# VacationPy by Paola VDW
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [54]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [55]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,1.01,99,100,1.94,GS,1719601961
1,1,port-aux-francais,-49.3500,70.2167,1.77,83,95,20.65,TF,1719601962
2,2,invercargill,-46.4000,168.3500,7.79,96,34,1.79,NZ,1719601962
3,3,mount gambier,-37.8333,140.7667,9.07,92,100,3.23,AU,1719601963
4,4,tazovsky,67.4667,78.7000,5.22,78,100,8.01,RU,1719601964


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [56]:
#%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    alpha=0.5, # Adjust the alpha value for transparency
    hover_cols = ["City"],    
    xlabel = "x",
    ylabel = "y"
)

# Display the map
map_plot_1 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [57]:
city_data_df.count()

City_ID       542
City          542
Lat           542
Lng           542
Max Temp      542
Humidity      542
Cloudiness    542
Wind Speed    542
Country       537
Date          542
dtype: int64

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [58]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df['Max Temp']>=25) & (city_data_df['Cloudiness']<50) & (city_data_df['Humidity']<=50)]

# Drop any rows with null values
ideal_weather_df.dropna(how='any')

# Display sample data
ideal_weather_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,jamestown,42.0970,-79.2353,25.77,49,0,4.47,US,1719601967
17,17,hamilton,39.1834,-84.5333,33.19,50,0,3.09,US,1719601972
47,47,ghadamis,30.1337,9.5007,39.83,10,0,3.37,LY,1719601988
49,49,bethel,41.3712,-73.4140,25.66,39,0,2.06,US,1719601863
59,59,ha'il,27.5219,41.6907,36.01,12,0,3.09,SA,1719601995
95,95,djibo,14.1022,-1.6306,37.97,26,38,0.64,BF,1719602017
106,106,guercif,34.2257,-3.3536,27.85,27,0,8.40,MA,1719602024
145,145,conceicao das alagoas,-19.9147,-48.3883,31.46,20,14,3.68,BR,1719602050
152,152,samalut,28.3121,30.7101,34.45,23,0,7.93,EG,1719602055
180,180,qarchi gak,37.0400,66.7889,31.66,28,0,8.33,AF,1719602083


### Step 3: Create a new DataFrame called `hotel_df`.

In [59]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df[['City','Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, 'Hotel Name', '')

# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
8,jamestown,US,42.0970,-79.2353,49,
17,hamilton,US,39.1834,-84.5333,50,
47,ghadamis,LY,30.1337,9.5007,10,
49,bethel,US,41.3712,-73.4140,39,
59,ha'il,SA,27.5219,41.6907,12,
95,djibo,BF,14.1022,-1.6306,26,
106,guercif,MA,34.2257,-3.3536,27,
145,conceicao das alagoas,BR,-19.9147,-48.3883,20,
152,samalut,EG,28.3121,30.7101,23,
180,qarchi gak,AF,37.0400,66.7889,28,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [60]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
# filters = f"circle:{hotel_df['Lng']}, {hotel_df['Lat']},{radius}"
# bias = f"proximity:{hotel_df['Lng']}, {hotel_df['Lat']}"
#limit = 20

params = {
    "categories":categories,
    #"limit":limit,
    # "filter":filters,
    # "bias":bias,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    long = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{long},{lat},{radius}"
    params["bias"] =f"proximity:{long},{lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address_request = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address_request.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jamestown - nearest hotel: DoubleTree Jamestown
hamilton - nearest hotel: North Vista Manor
ghadamis - nearest hotel: فندق عين الفارس
bethel - nearest hotel: Hampton Inn Danbury
ha'il - nearest hotel: فندق كلاسيك
djibo - nearest hotel: No hotel found
guercif - nearest hotel: No hotel found
conceicao das alagoas - nearest hotel: No hotel found
samalut - nearest hotel: No hotel found
qarchi gak - nearest hotel: No hotel found
ocean city - nearest hotel: Watson's Regency Suites
seydi - nearest hotel: No hotel found
mccook - nearest hotel: Coratel Inn & Suites By Jasper McCook
arauquita - nearest hotel: Hotel Oiti
manhattan - nearest hotel: The Mark
palmer - nearest hotel: No hotel found
ad dabbah - nearest hotel: No hotel found
kayenta - nearest hotel: Wetherill Inn
metlili chaamba - nearest hotel: No hotel found
qadirpur ran - nearest hotel: No hotel found
kabalo - nearest hotel: No hotel found
vilppula - nearest hotel: Rapukartano
el fasher - nearest hotel: Noor Al

,City,Country,Lat,Lng,Humidity,Hotel Name
8,jamestown,US,42.0970,-79.2353,49,DoubleTree Jamestown
17,hamilton,US,39.1834,-84.5333,50,North Vista Manor
47,ghadamis,LY,30.1337,9.5007,10,فندق عين الفارس
49,bethel,US,41.3712,-73.4140,39,Hampton Inn Danbury
59,ha'il,SA,27.5219,41.6907,12,فندق كلاسيك
95,djibo,BF,14.1022,-1.6306,26,No hotel found
106,guercif,MA,34.2257,-3.3536,27,No hotel found
145,conceicao das alagoas,BR,-19.9147,-48.3883,20,No hotel found
152,samalut,EG,28.3121,30.7101,23,No hotel found
180,qarchi gak,AF,37.0400,66.7889,28,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [61]:
#%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    alpha=0.5, # Adjust the alpha value for transparency
    hover_cols = ["City", "Hotel Name", "Country"],    
    xlabel = "x",
    ylabel = "y"
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)